In [1]:
import torch
import sys
sys.path.append("/home/sbalakri/PycharmProjects/PhD_1st_sem/e2cnn")
from e2cnn import gspaces
from e2cnn import nn
from matplotlib import pyplot as plt
hidden_channels = 1

In [2]:
class C8SteerableCNN(torch.nn.Module):
    
    def __init__(self, n_classes=10):
        
        super(C8SteerableCNN, self).__init__()
        
        # the model is equivariant under rotations by 45 degrees, modelled by C8
        self.r2_act = gspaces.Rot2dOnR2(N=8)
        
        # the input image is a scalar field, corresponding to the trivial representation
        in_type = nn.FieldType(self.r2_act, [self.r2_act.trivial_repr])
        
        # we store the input type for wrapping the images into a geometric tensor during the forward pass
        self.input_type = in_type
        
        # convolution encoder 1
        # first specify the output type of the convolutional layer
        # we choose 16 feature fields, each transforming under the regular representation of C8
        out_type = nn.FieldType(self.r2_act, 16*[self.r2_act.regular_repr])
        self.block1 = nn.SequentialModule(
            nn.R2Conv(in_type, out_type, kernel_size=3, padding=1, maximum_offset=0),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type, inplace=True)
        )
        
        # convolution encoder 2
        # the old output type is the input type to the next layer
        in_type = self.block1.out_type
        # the output type of the second convolution layer are 32 regular feature fields of C8
        out_type = nn.FieldType(self.r2_act, 8*[self.r2_act.regular_repr])
        self.block2 = nn.SequentialModule(
            nn.R2Conv(in_type, out_type, kernel_size=3, padding=1, maximum_offset=0),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type, inplace=True)
        )
        self.pool1 = nn.SequentialModule(
            nn.PointwiseMaxPool(out_type, kernel_size=3, padding=1, stride=2)
        )
        
        # convolution encoder 3
        # the old output type is the input type to the next layer
        in_type = self.block2.out_type
        # the output type of the third convolution layer are 32 regular feature fields of C8
        out_type = nn.FieldType(self.r2_act, 4*[self.r2_act.regular_repr])
        self.block3 = nn.SequentialModule(
            nn.R2Conv(in_type, out_type, kernel_size=3, padding=1, maximum_offset=0),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type, inplace=True)
        )
        
        # convolution encoder 4
        # the old output type is the input type to the next layer
        in_type = self.block3.out_type
        # the output type of the fourth convolution layer are 64 regular feature fields of C8
        out_type = nn.FieldType(self.r2_act, 1*[self.r2_act.regular_repr])
        self.block4 = nn.SequentialModule(
            nn.R2Conv(in_type, out_type, kernel_size=3, padding=1, maximum_offset=0),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type, inplace=True)
        )
        self.pool2 = nn.SequentialModule(
            nn.PointwiseMaxPool(out_type, kernel_size=3, padding=1, stride=4)
        )
        
        self.input_type1 = self.block4.out_type
                
        # convolution decoder 1 block 1
        # the old output type is the input type to the next layer
        in_type = self.block4.out_type
        # The output type of the third convolution layer are 32 regular feature fields of C8
        out_type = nn.FieldType(self.r2_act, 4*[self.r2_act.regular_repr])
        self.block3_dec = nn.SequentialModule(
            nn.R2Conv(in_type, out_type, kernel_size=3, padding=2, stride=1),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type, inplace=True)
        )
        
        
        # convolution ddecoder 1 block 2 
        # the old output type is the input type to the next layer
        in_type = self.block3_dec.out_type
        # the output type of the fourth convolution layer are 64 regular feature fields of C8
        out_type = nn.FieldType(self.r2_act, 8*[self.r2_act.regular_repr])
        self.block4_dec = nn.SequentialModule(
            nn.R2Conv(in_type, out_type, kernel_size=3, padding=1, stride=1),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type, inplace=True)
        )
        self.pool2_dec = nn.SequentialModule(
            nn.R2Upsampling(out_type, scale_factor=5/2)
        )
        
        
       # convolution decoder 2 block 1
        # first specify the output type of the convolutional layer
        # we choose 16 feature fields, each transforming under the regular representation of C8
        in_type = self.block4_dec.out_type
        out_type = nn.FieldType(self.r2_act, 16*[self.r2_act.regular_repr])
        self.block1_dec = nn.SequentialModule(
            nn.R2Conv(in_type, out_type, kernel_size=3, padding=1, stride=1),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type, inplace=True)
        )
        
        # convolution decoder 2 block 2
        # the old output type is the input type to the next layer
        in_type = self.block1_dec.out_type
        # the output type of the second convolution layer are 32 regular feature fields of C8
        out_type = nn.FieldType(self.r2_act, [self.r2_act.trivial_repr])
        self.block2_dec = nn.SequentialModule(
            nn.R2Conv(in_type, out_type, kernel_size=3, padding=1, stride=1),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type, inplace=True)
        )
     
        self.pool1_dec = nn.SequentialModule(
            nn.R2Upsampling(out_type, scale_factor=29/15)
        )
        

    def encode(self, input: torch.Tensor):
        
        x = nn.GeometricTensor(input, self.input_type)
        x = self.block1(x)
        x = self.block2(x)
        x = self.pool1(x)
        
        x = self.block3(x)
        x = self.block4(x)
        x = self.pool2(x)
        
        return x 

    def decode(self, input: torch.Tensor):
        
        x = nn.GeometricTensor(input, self.input_type1)
        x = self.block3_dec(x)
        x = self.block4_dec(x)
        x = self.pool2_dec(x)
        
        x = self.block1_dec(x)
        x = self.block2_dec(x)
        x = self.pool1_dec(x)
    
        return x 
    
    
    def forward(self, input: torch.Tensor):
        # wrap the input tensor in a GeometricTensor
        # (associate it with the input type)
        x = nn.GeometricTensor(input, self.input_type)
        
        # apply each equivariant block
        
        # Each layer has an input and an output type
        # A layer takes a GeometricTensor in input.
        # This tensor needs to be associated with the same representation of the layer's input type
        #
        # The Layer outputs a new GeometricTensor, associated with the layer's output type.
        # As a result, consecutive layers need to have matching input/output types
        x = self.block1(x)
        x = self.block2(x)
        x = self.pool1(x)
        
        x = self.block3(x)
        x = self.block4(x)
        x = self.pool2(x)        

        x = self.block3_dec(x)
        x = self.block4_dec(x)
        x = self.pool2_dec(x)
        
        x = self.block1_dec(x)
        x = self.block2_dec(x)
        x = self.pool1_dec(x)

        
        # pool over the group
#         x = self.gpool(x)

        # unwrap the output GeometricTensor
        # (take the Pytorch tensor and discard the associated representation)
        x = x.tensor
        
        # classify with the final fully connected layers)
#         x = self.fully_net(x.reshape(x.shape[0], -1))
        
        return x

In [3]:
# download the dataset
!wget -nc http://www.iro.umontreal.ca/~lisa/icml2007data/mnist_rotation_new.zip
# uncompress the zip file
!unzip -n mnist_rotation_new.zip -d mnist_rotation_new

File ‘mnist_rotation_new.zip’ already there; not retrieving.

Archive:  mnist_rotation_new.zip


In [4]:
from torch.utils.data import Dataset
from torchvision.transforms import RandomRotation
from torchvision.transforms import Pad
from torchvision.transforms import ToTensor
from torchvision.transforms import Compose

import numpy as np

from PIL import Image

device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [5]:
class MnistRotDataset(Dataset):
    
    def __init__(self, mode, transform=None):
        assert mode in ['train', 'test']
            
        if mode == "train":
            file = "mnist_rotation_new/mnist_all_rotation_normalized_float_train_valid.amat"
        else:
            file = "mnist_rotation_new/mnist_all_rotation_normalized_float_test.amat"
        
        self.transform = transform

        data = np.loadtxt(file, delimiter=' ')
        if mode == "train":    
            self.images = data[:, :-1][:500].reshape(-1, 28, 28).astype(np.float32)
            self.labels = data[:, -1][:500].astype(np.int64)
        else:
            self.images = data[:, :-1][:30000].reshape(-1, 28, 28).astype(np.float32)
            self.labels = data[:, -1][:30000].astype(np.int64)
        self.num_samples = len(self.labels)
    
    def __getitem__(self, index):
        image, label = self.images[index], self.labels[index]
        image = Image.fromarray(image)
        if self.transform is not None:
            image = self.transform(image)
#         import pdb; pdb.set_trace()
        return image, label
    
    def __len__(self):
        return len(self.labels)

# images are padded to have shape 29x29.
# this allows to use odd-size filters with stride 2 when downsampling a feature map in the model
pad = Pad((0, 0, 1, 1), fill=0)
totensor = ToTensor()

In [6]:
model = C8SteerableCNN().to(device)
if torch.cuda.is_available():
    model = model.cuda()


/opt/conda/conda-bld/pytorch_1573049308701/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1573049308701/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1573049308701/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/opt/conda/conda-bld/pytorch_1573049308701/work/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.


In [7]:
torch.cuda.is_available()# from torchsummary import summary
# summary(model, (1, 29, 29))

False

In [8]:
def test_model(model: torch.nn.Module, x: Image):
    # evaluate the `model` on 8 rotated versions of the input image `x`
    model.eval()
    
    x = pad(x)
    
    print(x)
    print('##########################################################################################')
    header = 'angle |  ' + '  '.join(["{:6d}".format(d) for d in range(10)])
    print(header)
    with torch.no_grad():
        for r in range(8):
            x_transformed = totensor(x.rotate(r*45., Image.BILINEAR)).reshape(1, 1, 29, 29)
            x_transformed.to(device)

            y = model(x_transformed)
            y = y.to('cpu').numpy().squeeze()
            
            angle = r * 45
            print("{:5d} : {}".format(angle, y))
    print('##########################################################################################')
    print()

    
# build the test set    
mnist_test = MnistRotDataset(mode='test')

# retrieve the first image from the test set
x, y = next(iter(mnist_test))

# evaluate the model
# test_model(model, x)

In [9]:
root = './data'
import os
pad = Pad((0, 0, 1, 1), fill=0)
totensor = ToTensor()
if not os.path.exists(root):
    os.mkdir(root)
import torchvision.datasets as dset
import torchvision.transforms as transforms
trans = transforms.Compose([
    pad,
    totensor,
])

# mnist_train = MnistRotDataset(mode='train', transform=train_transform)
# train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=32)
# if not exist, download mnist dataset
train_set = dset.MNIST(root=root, train=True, transform=trans, download=True)

train_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=16,
                 shuffle=True)

test_transform = Compose([
    pad,
    totensor,
])
mnist_test = MnistRotDataset(mode='test', transform=test_transform)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=16)

loss_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.00001)

In [10]:
import time
t0 = time.time()
loss_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.00001)
epoch = 0
for epoch in range(1):
    model.train()
    running_loss = 0
    num_samples = 0
    for i, (x, t) in enumerate(train_loader):
        
        if i>10:
            break
        
        optimizer.zero_grad()
        y = model(x)
        loss = loss_function(y, x)
        loss.backward()
        optimizer.step()   
        
        running_loss += loss.item()
        num_samples = num_samples + 1
    if epoch % 5 == 0:   # print every 2000 mini-batches
        print('[%d, %5d] loss: %.20f' %
              (epoch + 1, 0 , running_loss / num_samples))
              
    print(epoch, running_loss / (num_samples*32))
    
print('{} seconds'.format(time.time() - t0))

[1,     0] loss: 0.29277355562556872659
0 0.009149173613299023
6.215287923812866 seconds


In [11]:

# t0 = time.time()
# num_samples = 0
# epoch = 0
# running_loss = 0
# for i, (x, t) in enumerate(test_loader):
#     y = model(x)
#     loss = loss_function(y, x)

#     running_loss += loss.item()
#     num_samples = num_samples + 1
    
# print('[%d, %5d] loss: %.20f' %
#       (1, 0 , running_loss / num_samples))
# running_loss = 0.0


# print('{} seconds'.format(time.time() - t0))

In [12]:
# other_mnist_test = MnistRotDataset(mode='test')
# x_image, y_label = other_mnist_test.__getitem__(5)

# x = pad(x_image)
# x_transformed = totensor(x).reshape(1, 1, 29, 29)
# y = model(x_transformed)

# dd = x_transformed.detach().numpy()[0][0]
# prediction = y.detach().numpy()
# kk = prediction[0][0]

In [13]:
enc_rep = torch.tensor([])
t_all = torch.tensor([])
for i, (x, t) in enumerate(train_loader):
    
    y = model.encode(x)
    y = y.tensor
    enc_rep=torch.cat((enc_rep, y), 0)

    t = t.float()
    t_all = torch.cat((t_all, t), 0)
    if i > 200:
        break


In [14]:
# for i, (x, t) in enumerate(test_loader):

#     y = model.encode(x)
#     y = y.tensor
#     decoded = model.decode(y)
#     if i > 2:
#         decoded = decoded.tensor
#         decoded1 = decoded.detach().numpy()

#         plt.imshow(decoded1[20].reshape(29,29))
#         plt.show()
#         plt.imshow(x[20].reshape(29,29))
#         break
 


In [16]:
enc_rep = enc_rep.reshape(-1, 1 * 8, 16)
encoded_rep_all = torch.tensor([])
for s in range(enc_rep.shape[0]):   
    encoded_rep = enc_rep[s]
    all_encs =  torch.tensor([])
    for m in range(hidden_channels):
        sum_enc_rep = 0
        for i in range(8):
            sum_enc_rep = sum_enc_rep + encoded_rep[8 * m + i]
        all_encs = torch.cat((all_encs, sum_enc_rep.reshape(1, 16)), 0)
        all_encs = all_encs/8.
    encoded_rep_all = torch.cat((encoded_rep_all, all_encs.reshape(1, all_encs.shape[0], all_encs.shape[1])), 0)


In [17]:
# Export  
enc_rep_all = encoded_rep_all.reshape(-1, hidden_channels*16).detach().numpy()
import pickle
features_path = "conv_autoe_features_train.pickle"
labels_path = "conv_autoe_labels_train.pickle"
orig_path = "conv_autoe_orig_features_train.pickle"
pickle.dump(enc_rep_all, open(features_path, 'wb'))

t_all1 = t_all.detach().numpy().reshape(t_all.shape[0], 1)
pickle.dump(t_all1, open(labels_path, 'wb'))

In [29]:
enc_rep = enc_rep.reshape(-1, 1*8, 16)
encoded_rep_all_sum = torch.tensor([])
for s in range(enc_rep.shape[0]):   
    encoded_rep = enc_rep[s]
    all_encs =  torch.tensor([])
    sum_enc_rep = torch.empty(16, 16, dtype=torch.float)
    m = 0
    for i in range(16):
        for j in range(16):
        #     for i in range(hidden_channels - mm):
            sum_enc_rep[i, j] = torch.dot(encoded_rep[m][i], encoded_rep[m][j]) + torch.dot(encoded_rep[m+1][i],encoded_rep[m+1][j]) + torch.dot(encoded_rep[m+2][i], encoded_rep[m+2][j]) + torch.dot(encoded_rep[m+3][i], encoded_rep[m+3][j]) + torch.dot(encoded_rep[m+4][i], encoded_rep[m+4][j]) + torch.dot(encoded_rep[m+5][i],encoded_rep[m+5][j]) + torch.dot(encoded_rep[m+6][i],encoded_rep[m+6][j]) + torch.dot(encoded_rep[m+7][i],encoded_rep[m+7][j])
#             sum_enc_rep[i+mm, mm] = sum_enc_rep[mm, i+mm]  
    encoded_rep_all_sum = torch.cat((encoded_rep_all_sum, sum_enc_rep.reshape(1, 16, 16)), 0)

RuntimeError: 1D tensors expected, got 0D, 0D tensors at /opt/conda/conda-bld/pytorch_1573049308701/work/aten/src/TH/generic/THTensorEvenMoreMath.cpp:774

In [31]:
torch.dot(encoded_rep[m][i], encoded_rep[m][j])

tensor(0., grad_fn=<SelectBackward>)

In [ ]:
enc_rep_all = sum_enc_rep.reshape(-1, 16*16).detach().numpy()
features_path = "conv_autoe_features_train_gram.pickle"
labels_path = "conv_autoe_labels_train_gram.pickle"
orig_path = "conv_autoe_orig_features_train_gram.pickle"
pickle.dump(enc_rep_all, open(features_path, 'wb'))

t_all1 = t_all.detach().numpy().reshape(t_all.shape[0], 1)
pickle.dump(t_all1, open(labels_path, 'wb'))

In [18]:
enc_rep = torch.tensor([])
t_all = torch.tensor([])
for i, (x, t) in enumerate(test_loader):
    
    y = model.encode(x)
    y = y.tensor
    enc_rep=torch.cat((enc_rep, y), 0)

    t = t.float()
    t_all = torch.cat((t_all, t), 0)


In [19]:
enc_rep = enc_rep.reshape(-1, 1 * 8, 16)
encoded_rep_all = torch.tensor([])
for s in range(enc_rep.shape[0]):   
    encoded_rep = enc_rep[s]
    all_encs =  torch.tensor([])
    for m in range(hidden_channels):
        sum_enc_rep = 0
        for i in range(8):
            sum_enc_rep = sum_enc_rep + encoded_rep[8 * m + i]
        all_encs = torch.cat((all_encs, sum_enc_rep.reshape(1, 16)), 0)
        all_encs = all_encs/8.
    encoded_rep_all = torch.cat((encoded_rep_all, all_encs.reshape(1, all_encs.shape[0], all_encs.shape[1])), 0)

In [20]:
# Export  
enc_rep_all = encoded_rep_all.reshape(-1, 16).detach().numpy()
import pickle
features_path = "conv_autoe_features_test.pickle"
labels_path = "conv_autoe_labels_test.pickle"
orig_path = "conv_autoe_orig_features_test.pickle"
pickle.dump(enc_rep_all, open(features_path, 'wb'))

t_all1 = t_all.detach().numpy().reshape(t_all.shape[0], 1)
pickle.dump(t_all1, open(labels_path, 'wb'))

In [ ]:
enc_rep = enc_rep.reshape(-1, 1*8, 16)
encoded_rep_all_sum = torch.tensor([])
for s in range(enc_rep.shape[0]):   
    encoded_rep = enc_rep[s]
    all_encs =  torch.tensor([])
    sum_enc_rep = torch.empty(16, 16, dtype=torch.float)
    m = 0
    for i in range(16):
        for j in range(16):
        #     for i in range(hidden_channels - mm):
            sum_enc_rep[i, j] = torch.dot(encoded_rep[m][i], encoded_rep[m][j]) + torch.dot(encoded_rep[m+1][i],encoded_rep[m+1][j]) + torch.dot(encoded_rep[m+2][i], encoded_rep[m+2][j]) + torch.dot(encoded_rep[m+3][i], encoded_rep[m+3][j]) + torch.dot(encoded_rep[m+4][i], encoded_rep[m+4][j]) + torch.dot(encoded_rep[m+5][i],encoded_rep[m+5][j]) + torch.dot(encoded_rep[m+6][i],encoded_rep[m+6][j]) + torch.dot(encoded_rep[m+7][i],encoded_rep[m+7][j])
#             sum_enc_rep[i+mm, mm] = sum_enc_rep[mm, i+mm]
    encoded_rep_all_sum = torch.cat((encoded_rep_all_sum, sum_enc_rep.reshape(1, 16, 16)), 0)

In [ ]:
enc_rep_all = sum_enc_rep.reshape(-1, 16*16).detach().numpy()
features_path = "conv_autoe_features_test_gram.pickle"
labels_path = "conv_autoe_labels_test_gram.pickle"
orig_path = "conv_autoe_orig_features_test_gram.pickle"
pickle.dump(enc_rep_all, open(features_path, 'wb'))

t_all1 = t_all.detach().numpy().reshape(t_all.shape[0], 1)
pickle.dump(t_all1, open(labels_path, 'wb'))

In [23]:
# #gram pooling
# enc_rep = enc_rep.reshape(-1, hidden_channels*8, 49)
# encoded_rep_all_sum = torch.tensor([])
# for s in range(enc_rep.shape[0]):   
#     encoded_rep = enc_rep[s]
#     all_encs =  torch.tensor([])
#     sum_enc_rep = torch.empty(hidden_channels, hidden_channels, dtype=torch.float)
#     for mm in range(hidden_channels):
#         m = 8 * mm
#         for i in range(hidden_channels - mm):
#             sum_enc_rep[mm, i+mm] = torch.dot(encoded_rep[m], encoded_rep[8*i + m]) + torch.dot(encoded_rep[m+1],encoded_rep[8*i + m+1]) + torch.dot(encoded_rep[m+2], encoded_rep[8*i + m+2]) + torch.dot(encoded_rep[m+3], encoded_rep[8*i + m+3]) + torch.dot(encoded_rep[m+4], encoded_rep[8*i + m+4]) + torch.dot(encoded_rep[m+5],encoded_rep[8*i + m+5]) + torch.dot(encoded_rep[m+6],encoded_rep[8*i + m+6]) + torch.dot(encoded_rep[m+7],encoded_rep[8*i + m+7])
#             sum_enc_rep[i+mm, mm] = sum_enc_rep[mm, i+mm]
#     encoded_rep_all_sum = torch.cat((encoded_rep_all_sum, sum_enc_rep.reshape(1, hidden_channels*hidden_channels)), 0)


In [24]:
#gram pooling
# enc_rep = enc_rep.reshape(-1, 1*8* 16)
# encoded_rep_all_sum = torch.tensor([])
# for s in range(enc_rep.shape[0]):   
#     encoded_rep = enc_rep[s]
#     all_encs =  torch.tensor([])
#     sum_enc_rep = torch.empty(hidden_channels, hidden_channels, dtype=torch.float)
#     for mm in range(hidden_channels):
#         m = 8 * mm
#         for i in range(hidden_channels - mm):
#             sum_enc_rep[mm, i+mm] = torch.dot(encoded_rep[m], encoded_rep[8*i + m]) + torch.dot(encoded_rep[m+1],encoded_rep[8*i + m+1]) + torch.dot(encoded_rep[m+2], encoded_rep[8*i + m+2]) + torch.dot(encoded_rep[m+3], encoded_rep[8*i + m+3]) + torch.dot(encoded_rep[m+4], encoded_rep[8*i + m+4]) + torch.dot(encoded_rep[m+5],encoded_rep[8*i + m+5]) + torch.dot(encoded_rep[m+6],encoded_rep[8*i + m+6]) + torch.dot(encoded_rep[m+7],encoded_rep[8*i + m+7])
#             sum_enc_rep[i+mm, mm] = sum_enc_rep[mm, i+mm]
#     encoded_rep_all_sum = torch.cat((encoded_rep_all_sum, sum_enc_rep.reshape(1, hidden_channels*hidden_channels)), 0)

RuntimeError: 1D tensors expected, got 0D, 0D tensors at /opt/conda/conda-bld/pytorch_1573049308701/work/aten/src/TH/generic/THTensorEvenMoreMath.cpp:774

In [ ]:
# enc_rep = enc_rep.reshape(-1, 1*8, 16)
# encoded_rep_all_sum = torch.tensor([])
# for s in range(enc_rep.shape[0]):   
#     encoded_rep = enc_rep[s]
#     all_encs =  torch.tensor([])
#     sum_enc_rep = torch.empty(16, 16, dtype=torch.float)
#     m = 0
#     for i in range(16):
#         for j in range(16):
#         #     for i in range(hidden_channels - mm):
#             sum_enc_rep[i, j] = torch.dot(encoded_rep[m][i], encoded_rep[m][j]) + torch.dot(encoded_rep[m+1][i],encoded_rep[m+1][j]) + torch.dot(encoded_rep[m+2][i], encoded_rep[m+2][j]) + torch.dot(encoded_rep[m+3][i], encoded_rep[m+3][j]) + torch.dot(encoded_rep[m+4][i], encoded_rep[m+4][j]) + torch.dot(encoded_rep[m+5][i],encoded_rep[m+5][j]) + torch.dot(encoded_rep[m+6][i],encoded_rep[m+6][j]) + torch.dot(encoded_rep[m+7][i],encoded_rep[m+7][j])
# #             sum_enc_rep[i+mm, mm] = sum_enc_rep[mm, i+mm]
# #     encoded_rep_all_sum = torch.cat((encoded_rep_all_sum, sum_enc_rep.reshape(1, 16, 16)), 0)

In [ ]:
# enc_rep_all = sum_enc_rep.reshape(-1, 16*16).detach().numpy()
# features_path = "conv_autoe_features_test_gram.pickle"
# labels_path = "conv_autoe_labels_test_gram.pickle"
# orig_path = "conv_autoe_orig_features_test_gram.pickle"
# pickle.dump(enc_rep_all, open(features_path, 'wb'))

# t_all1 = t_all.detach().numpy().reshape(t_all.shape[0], 1)
# pickle.dump(t_all1, open(labels_path, 'wb'))

In [ ]:
# enc_rep_all = sum_enc_rep.reshape(-1, hidden_channels*hidden_channels).detach().numpy()
# features_path = "conv_autoe_features_train_gram.pickle"
# labels_path = "conv_autoe_labels_train_gram.pickle"
# orig_path = "conv_autoe_orig_features_train_gram.pickle"
# pickle.dump(enc_rep_all, open(features_path, 'wb'))

# t_all1 = t_all.detach().numpy().reshape(t_all.shape[0], 1)
# pickle.dump(t_all1, open(labels_path, 'wb'))

In [ ]:
sum_enc_rep.shape